#**Import Modules**

Selenium, Openpyxl, Regex, Files, Time

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 5.2MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *
from selenium.webdriver.common.action_chains import ActionChains

In [0]:
import time

In [0]:
import openpyxl

In [0]:
import regex as re

In [0]:
from google.colab import files

#**Mount Google Drive**

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [16]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Fetch all Paper Links**

In [17]:
MAINURL = "https://www.kci.go.kr/kciportal/po/search/poArtiSearList.kci?sereId=001119"
articleUrls = []

# access main page
wd.get(MAINURL)
time.sleep(0.5)

# fetch last page number
try:
    LASTPAGENO = wd.find_element_by_xpath("//a[@href]/img[@alt='마지막']").find_element_by_xpath("..").get_attribute("href")
    LASTPAGENO = int(LASTPAGENO[LASTPAGENO.index("(") + 1:LASTPAGENO.index(")")])
except NoSuchElementException:
    print("NO LAST PAGE!")

# fetch article urls
currPage = 1

while currPage <= LASTPAGENO:
    print("Reading page : " + str(currPage))

    # fetch all articles
    articles = wd.find_elements_by_xpath("//table[@class='list_paper no_title']/tbody/tr/td/p/label/a[@href]")
    for article in articles:
        articleUrls.append(article.get_attribute("href"))

    # go to next page
    try:
        TARGET = "//div[@class='pagenate']/a[@href=\'javascript:goPage(" + str(currPage + 1) + ")\']"
        wd.find_element_by_xpath(TARGET).click()
        currPage += 1
        time.sleep(0.8)
    except NoSuchElementException:
        break

len(articleUrls)

Reading page : 1
Reading page : 2
Reading page : 3
Reading page : 4
Reading page : 5
Reading page : 6
Reading page : 7
Reading page : 8
Reading page : 9
Reading page : 10
Reading page : 11
Reading page : 12
Reading page : 13
Reading page : 14
Reading page : 15
Reading page : 16
Reading page : 17
Reading page : 18
Reading page : 19
Reading page : 20
Reading page : 21
Reading page : 22
Reading page : 23
Reading page : 24
Reading page : 25
Reading page : 26
Reading page : 27
Reading page : 28
Reading page : 29
Reading page : 30
Reading page : 31


304

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
TARGETPATH = "./Data/raw/Korean/"

In [32]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'journal'
 ws.cell(row = 1, column = 2).value = 'year'
 ws.cell(row = 1, column = 3).value = 'title'
 ws.cell(row = 1, column = 4).value = 'keywords'
 ws.cell(row = 1, column = 5).value = 'authors'

 for articleurl in articleUrls:
    # access volume page
    wd.get(articleurl)
    time.sleep(1)

    # fetch title
    try:
        title = wd.find_element_by_xpath("//p[@class='title']/span").get_attribute("innerText")
    except NoSuchElementException:
        continue
    #print(title)

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//div[@class='journalInfo']/p[@class='vol']/a").get_attribute("innerText").strip()
        yr = yr[:yr.index(",")]
    except ValueError:
        continue
    except NoSuchElementException:
        continue
    #print(yr)
        
    # fetch authors
    authors = []
    a_tags = wd.find_elements_by_xpath("//div[@class='author']/a")
    for a in a_tags:
        author = a.get_attribute("innerText").strip()
        author = author[:author.index(" ")]
        authors.append(author)
    while ' ' in authors:
        authors.remove(' ')
    while '' in authors:
        authors.remove('')
    if(len(authors) == 0):
        continue
    #print(authors)

    # fetch keywords
    keywords = []
    try:
        h2_tag = wd.find_element_by_xpath("//div[@class='articleBody']/div[@class='box']/h2[contains(text(), '키워드')]")
        try:
            p_tag = h2_tag.find_element_by_xpath("../div/p")
            try:
                a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
                keywords_candid_1 = p_tag.get_attribute("innerText").strip().split(", ")
                keywords_candid_2 = []
                for a in a_tags:
                    keywords_candid_2.append(a.get_attribute("innerText"))
                candid_1 = 0
                candid_2 = 0
                for kwd in keywords_candid_1:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_1 += 1
                for kwd in keywords_candid_2:
                    if len(re.compile("[가-힣]+").findall(kwd)) > 0:
                        candid_2 += 1
                if candid_1 <= candid_2:
                    keywords = keywords_candid_1
                else:
                    keywords = keywords_candid_2
            except NoSuchElementException:
                keywords = p_tag.get_attribute("innerText").strip().split(", ")
        except NoSuchElementException:
            a_tags = h2_tag.find_elements_by_xpath("../div/div/a")
            for a in a_tags:
                keywords.append(a.get_attribute("innerText"))
    except NoSuchElementException:
        continue
    while ' ' in keywords:
        keywords.remove(' ')
    while '' in keywords:
        keywords.remove('')
    '''
    hanguel_idx = []
    for i in range(len(keywords)):
        if len(re.compile("[가-힣]+").findall(keywords[i])) > 0:
            hanguel_idx.append(i)
    hanguel_idx.reverse()
    for i in hanguel_idx:
        del keywords[i]
    '''
    if(len(keywords) == 0):
        continue
    #print(keywords)

    # write data to excel file
    print("Writing, " + title)
    ws.cell(row = r, column = 1).value = "Journal of Korea Trade"
    ws.cell(row = r, column = 2).value = yr
    ws.cell(row = r, column = 3).value = title
    for j in range(len(keywords)):
        ws.cell(row = r + j, column = 4).value = keywords[j]
    for j in range(len(authors)):
        ws.cell(row = r + j, column = 5).value = authors[j]
    r += max([len(keywords), len(authors)])

    if articleUrls.index(articleurl) % 10 == 0:
        print(str(articleUrls.index(articleurl) + 1) + " out of " + str(len(articleUrls)))

Writing, Does FDI crowd out domestic firms? Micro-level evidence from the Republic of Korea
1 out of 304
Writing, Services in global value chains – the cases of Taiwan and Korea
Writing, Accounting of gross exports and tracing foreign values in the global pharmaceutical value chain - Where does Korea stand?
Writing, Trade-employment linkage of FTAs in Korea: Inter- and intra-industry reallocations of production resources
Writing, Court’s intervention on arbitral tribunal’s jurisdiction - From the perspective of the Korean Arbitration Act (2016)
Writing, The trade and income effects of RCEP: implications for China and Korea
Writing, The effects of Paris agreements on Korean economy and trade analyzed by the computational general equilibrium method considering firm’s heterogeneity
Writing, 국가 및 기업이미지가 미국소비자의 한국제품평가에 미치는 영향에 관한 연구
Writing, Developing value added service of cold chain logistics between China and Korea
Writing, Tariff barriers and industrial spillover effects
Writing, Promo

#**Save File**

In [0]:
wb.save(filename = TARGETPATH + "journal_of_korea_trade.xlsx")

#**Download FIle**

In [0]:
files.download(TARGETPATH + "journal_of_korea_trade.xlsx")